## Colophon: Notebook and Module

This is both a jupyter notebook and a python module, sync'd using [jupytext][].

[jupytext]: https://github.com/mwouts/jupytex

### OCap Discipline in Python

When loaded as a module, 
in order to support patterns of cooperation without vulnerability,
we maintain [OCap Discipline](https://github.com/dckc/awesome-ocap/wiki/DisciplinedPython)
by avoiding access to IO and other ambient authority,
except when loaded in the [top-level code environment](https://docs.python.org/3/library/__main__.html#what-is-the-top-level-code-environment), which we can test with `__name__ == '__main__'`.
Note that jupyter notebooks run in the top-level code environment.

In [1]:
import logging

log = logging.getLogger(__name__)

IO_TESTING = False

if __name__ == '__main__':
    from sys import version as sys_version, stderr

    IO_TESTING = True
    logging.basicConfig(level=logging.INFO, stream=stderr)
    log.info('python version: %s', dict(
        python=sys_version,
    ))

INFO:__main__:python version: {'python': '3.7.10 | packaged by conda-forge | (default, Feb 19 2021, 16:07:37) \n[GCC 9.3.0]'}


In [2]:
from urllib.request import Request, HTTPError, BaseHandler

if IO_TESTING:
    # Use leading _ to remind ourselves of ambient authorities
    # that should not be available when loaded as a module.
    def _build_opener():
        from urllib.request import build_opener
        return build_opener()

    _theWeb = _build_opener()
    _theWeb.cache = {}

## Cosmos / Tendermint RPC

The Agoric blockchain is based on Cosmos-SDK, which uses Tendermint consensus.
Each layer has an OpenAPI / Swagger / REST API with interactive explorer style documentation:

 - [Tendermint RPC](https://docs.tendermint.com/master/rpc/)
 - [Cosmos RPC](https://v1.cosmos.network/rpc/v0.44.5)


### Agoric xnet RPC server

The hostname of the RPC server we're exploring is:

In [3]:
RPC_HOST = 'xnet.rpc.agoric.net'

But if we try to access it using the python standard library defaults, we are forbidden:

In [4]:
if IO_TESTING:
    try:
        _theWeb.open(f'https://{RPC_HOST}/status?')
    except HTTPError as err:
        log.error(err)

ERROR:__main__:HTTP Error 403: Forbidden


So let's tell a little bit about who we are and what business we are about using `User-Agent` :

In [5]:
import json

def ua_req(url, user_agent='agoric.com test exploration'):
    return Request(url, headers={'User-Agent': user_agent})

def get_json(url, ua):
    return json.loads(ua.open(ua_req(url)).read())

IO_TESTING and get_json('http://httpbin.org/headers', _theWeb)

{'headers': {'Accept-Encoding': 'identity',
  'Host': 'httpbin.org',
  'User-Agent': 'agoric.com test exploration',
  'X-Amzn-Trace-Id': 'Root=1-62ff2747-0d3e8e6e2e5ef9a037a9b933'}}

Now we can start exploring:

In [6]:
IO_TESTING and get_json(f'https://{RPC_HOST}/status?', _theWeb)

{'jsonrpc': '2.0',
 'id': -1,
 'result': {'node_info': {'protocol_version': {'p2p': '8',
    'block': '11',
    'app': '0'},
   'id': '61d979eb255e925e0f68790662147e327234aab5',
   'listen_addr': 'tcp://0.0.0.0:26656',
   'network': 'agoricxnet-13',
   'version': '0.34.14',
   'channels': '40202122233038606100',
   'moniker': 'validator-0',
   'other': {'tx_index': 'on', 'rpc_address': 'tcp://0.0.0.0:26657'}},
  'sync_info': {'latest_block_hash': '8A24A78238E83E249040148387AB32E317F555AC9AE3EFC47D522B3950E030C9',
   'latest_app_hash': '949CA8CB8EE69088C920410C40BCF4890BDA8BF7A63EB082D5F0DCC8A6AB414D',
   'latest_block_height': '7338',
   'latest_block_time': '2022-08-19T06:01:36.013125837Z',
   'earliest_block_hash': 'EDD7A15ACD6CACA49F0ABE4567481EE0C778D1367B6B334215A61583CF7DBFD4',
   'earliest_app_hash': 'E3B0C44298FC1C149AFBF4C8996FB92427AE41E4649B934CA495991B7852B855',
   'earliest_block_height': '1',
   'earliest_block_time': '2022-08-18T18:38:05.02142049Z',
   'catching_up': Fal

### JSON Helper

Navigating JSON structured in python is a bit tedious using `obj['headers']` syntax.
So let's borrow `obj.headers` style from JavaScript.

In [7]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

def json_parse(s):
    return json.loads(s, object_hook=AttrDict)

def get_json(url, ua):
    # ISSUE: cache may get in the way of observing changing data
    cache = hasattr(ua, 'cache') and ua.cache
    if cache is not None and url in ua.cache:
        return ua.cache[url]
    # log.info('cache miss %s %s', not not cache, url)
    obj = json_parse(ua.open(ua_req(url)).read())
    if cache is not None:
        cache[url] = obj
    return obj

json_parse('{"headers": {"Host": "httpbin.org"}}').headers

{'Host': 'httpbin.org'}

## What Pools are on the Agoric AMM?

The AMM, like all Inter Protocol contracts, publishes metrics using via an `x/vstorage` module.

In [8]:
from base64 import b64decode


class StorageNode:
    """Query Agoric x/vstorage module"""
    def __init__(self, ua, path, base=f'https://{RPC_HOST}'):
        self._path = path
        self._base = base
        self.__ua = ua

    def __repr__(self):
        return self._path

    def _url(self, data=False):
        kind = 'data' if data else 'children'
        return f'{self._base}/abci_query?path=%22/custom/vstorage/{kind}/{self._path}%22&height=0'

    @property
    def _keys_url(self):
        return self._url()
            
    @property
    def _data_url(self):
        return self._url(True)

    def keys(self):
        tendermint_response = get_json(self._keys_url, self.__ua)
        # log.debug('tendermint response %s', tendermint_response)
        return json_parse(b64decode(tendermint_response.result.response.value)).children

    def __call__(self):
        tendermint_response = get_json(self._data_url, self.__ua)
        # log.debug('tendermint response %s', tendermint_response)
        return json_parse(b64decode(tendermint_response.result.response.value)).value

    def __getattr__(self, name):
        return self.__class__(self.__ua, f'{self._path}.{name}', self._base)

IO_TESTING and StorageNode(_theWeb, 'published').keys()

['agoricNames',
 'amm',
 'priceFeed',
 'psm',
 'reserve',
 'stakeFactory',
 'vaultFactory',
 'wallet']

In [9]:
IO_TESTING and StorageNode(_theWeb, 'published').amm.keys()

['governance', 'metrics', 'pool0']

In [10]:
IO_TESTING and StorageNode(_theWeb, 'published').amm.metrics()

'{"body":"{\\"XYK\\":[{\\"@qclass\\":\\"slot\\",\\"iface\\":\\"Alleged: IbcATOM brand\\",\\"index\\":0}]}","slots":["board03446"]}'

The format follows Agoric distributed objects conventions (specifically, `CapData` from `@endo/marshal`);
typically, it uses Agoric board IDs for slots:

In [11]:
class ObjectNode(StorageNode):
    """Query storage nodes using Distribute Objects conventions"""
    
    @classmethod
    def root(cls, ua):
        return cls(path='published', ua=ua)

    @classmethod
    def cleanup(cls, capData):
        return AttrDict({'body': json_parse(capData.body), 'slots': capData.slots})

    def __call__(self, marshaller=None):
        capData = json_parse(StorageNode.__call__(self))
        return marshaller.unserialize(capData) if marshaller else self.cleanup(capData)

if IO_TESTING:
    _amm = ObjectNode.root(_theWeb).amm

IO_TESTING and _amm.metrics()

{'body': {'XYK': [{'@qclass': 'slot',
    'iface': 'Alleged: IbcATOM brand',
    'index': 0}]},
 'slots': ['board03446']}

In [12]:
class Marshal:
    def __init__(self, convertSlotToVal=lambda slot, iface: slot):
        self.convertSlotToVal = convertSlotToVal

    def unserialize(self, capData):
        s2v = self.convertSlotToVal
        slots = capData.slots
        def object_hook(obj):
            qclass = obj.get('@qclass')
            if qclass == 'slot':
                # TODO? hilbert hotel
                index = obj['index']
                return s2v(slots[index], obj.get('iface'))
            elif qclass == 'bigint':
                return int(obj['digits'])
            return AttrDict(obj)
        return json.loads(capData.body, object_hook=object_hook)


class SlotCache:
    def __init__(self, slotKey):
        self._cache = {}
        self.slotKey = slotKey

    def convertSlotToVal(self, slot, iface):
        cache = self._cache
        if slot in cache:
            return cache[slot]
        obj = AttrDict({self.slotKey: slot, 'iface': iface})
        cache[slot] = obj
        return obj

if IO_TESTING:
    # Preserve identities when deserializing using boardIds
    _fromBoard = Marshal(SlotCache('boardId').convertSlotToVal)

IO_TESTING and _amm.metrics(_fromBoard)

{'XYK': [{'boardId': 'board03446', 'iface': 'Alleged: IbcATOM brand'}]}

The `XYK` member of the AMM metrics is a list of brands.
The format carries their _alleged_ name, for debugging,
but take care not to rely on it for anything else:

In [13]:
IO_TESTING and [brand.iface for brand in _amm.metrics(_fromBoard).XYK]

['Alleged: IbcATOM brand']

### AMM Governance

In [14]:
IO_TESTING and _amm.governance(_fromBoard)

{'current': {'Electorate': {'type': 'invitation',
   'value': {'brand': {'boardId': 'board03125',
     'iface': 'Alleged: Zoe Invitation brand'},
    'value': [{'description': 'questionPoser',
      'handle': {'boardId': 'board00126',
       'iface': 'Alleged: InvitationHandle'},
      'installation': {'boardId': 'board05311',
       'iface': 'Alleged: BundleInstallation'},
      'instance': {'boardId': 'board00613',
       'iface': 'Alleged: InstanceHandle'}}]}},
  'MinInitialPoolLiquidity': {'type': 'amount',
   'value': {'brand': {'boardId': 'board0074', 'iface': 'Alleged: IST brand'},
    'value': 0}},
  'PoolFee': {'type': 'nat', 'value': 24},
  'ProtocolFee': {'type': 'nat', 'value': 6}}}

### Brands from the `agoricNames` distinguished name hub

Agoric provides an authoritative collection of brands via the `agoricNames` name hub:

In [15]:
def fromEntries(entries):
    return AttrDict({key: val for key, val in entries})

class AgoricNames:
    # ISSUE: this treats contents as static when in theory, they could change
    def __init__(self, ua, marshaller):
        root = ObjectNode.root(ua)
        agoricNames = root.agoricNames
        log.info('agoricNames keys %s', agoricNames.keys())
        self.brand = fromEntries(agoricNames.brand(marshaller))
        self.oracleBrand = fromEntries(agoricNames.oracleBrand(marshaller))
        self.issuer = fromEntries(agoricNames.issuer(marshaller))
        self.installation = fromEntries(agoricNames.installation(marshaller))
        self.instance = fromEntries(agoricNames.instance(marshaller))

if IO_TESTING:
    _agoricNames = AgoricNames(_theWeb, _fromBoard)

IO_TESTING and _agoricNames.brand

INFO:__main__:agoricNames keys ['brand', 'installation', 'instance', 'issuer', 'oracleBrand', 'uiConfig']


{'BLD': {'boardId': 'board0592', 'iface': 'Alleged: BLD brand'},
 'IST': {'boardId': 'board0074', 'iface': 'Alleged: IST brand'},
 'Attestation': {'boardId': 'board05432',
  'iface': 'Alleged: Attestation brand'},
 'AUSD': {'boardId': 'board05815', 'iface': 'Alleged: AUSD brand'},
 'IbcATOM': {'boardId': 'board03446', 'iface': 'Alleged: IbcATOM brand'}}

Now we can test brands for identity:

In [16]:
IO_TESTING and _amm.metrics(_fromBoard).XYK[0] is _agoricNames.brand.IbcATOM

True

### PyData tools: Pandas DataFrames

In [17]:
import pandas as pd

log.info(dict(pandas=pd.__version__))

INFO:__main__:{'pandas': '1.3.2'}


In [18]:
IO_TESTING and pd.DataFrame.from_dict(_agoricNames.brand, orient='index').rename_axis('keyword')

,boardId,iface
keyword,,
BLD,board0592,Alleged: BLD brand
IST,board0074,Alleged: IST brand
Attestation,board05432,Alleged: Attestation brand
AUSD,board05815,Alleged: AUSD brand
IbcATOM,board03446,Alleged: IbcATOM brand


## Status of 1st AMM pool (`pool0`)

In [19]:
IO_TESTING and _amm.pool0.metrics(_fromBoard)

{'centralAmount': {'brand': {'boardId': 'board0074',
   'iface': 'Alleged: IST brand'},
  'value': 200053028931},
 'liquidityTokens': {'brand': {'boardId': 'board06445',
   'iface': 'Alleged: IbcATOMLiquidity brand'},
  'value': 200007998503},
 'secondaryAmount': {'brand': {'boardId': 'board03446',
   'iface': 'Alleged: IbcATOM brand'},
  'value': 99981552}}

In [20]:
from decimal import Decimal

def amt(nameHub):
    def aux(obj, unit=1000000):
        found = [n for n, v in nameHub.items() if v is obj.brand]
        if found:
            [brand] = found
        else:
            log.warning('unknown brand %s', obj.brand)
            brand = obj.brand.iface
        amount = Decimal(obj.value) / unit
        return pd.Series(dict(amount=amount, brand=brand))
    return aux

if IO_TESTING:
    _amt = amt(_agoricNames.brand)

IO_TESTING and _amt(_amm.pool0.metrics(_fromBoard).centralAmount)

amount    200053.028931
brand               IST
dtype: object

In [21]:
IO_TESTING and pd.DataFrame([pd.Series(_amt(a), name=col)
                             for col, a in _amm.pool0.metrics(_fromBoard).items()])

,amount,brand
centralAmount,200053.028931,IST
liquidityTokens,200007.998503,Alleged: IbcATOMLiquidity brand
secondaryAmount,99.981552,IbcATOM


## Price Feed: ATOM in USD

In [29]:
if IO_TESTING:
    _priceFeed = ObjectNode.root(_theWeb).priceFeed

IO_TESTING and _priceFeed.keys()

['ATOM_USD_price_feed']

In [33]:
IO_TESTING and _priceFeed.ATOM_USD_price_feed(_fromBoard)

{'quoteAmount': {'brand': {'boardId': None,
   'iface': 'Alleged: InvitationHandle'},
  'value': [{'amountIn': {'brand': {'boardId': 'board02810',
      'iface': 'Alleged: ATOM brand'},
     'value': 1},
    'amountOut': {'brand': {'boardId': 'board0639',
      'iface': 'Alleged: USD brand'},
     'value': 19},
    'timer': {'boardId': None, 'iface': 'Alleged: InvitationHandle'},
    'timestamp': 1660853344}]},
 'quotePayment': {'boardId': None, 'iface': 'Alleged: InvitationHandle'}}

## Vaults (0 vaults so far)

In [22]:
if IO_TESTING:
    _vaultFactory = ObjectNode.root(_theWeb).vaultFactory

IO_TESTING and _vaultFactory.keys()

['governance', 'manager0', 'metrics', 'timingParams']

In [23]:
IO_TESTING and _vaultFactory.manager0.keys()

['governance', 'metrics']

In [25]:
IO_TESTING and _vaultFactory.manager0.metrics(_fromBoard)

{'numActiveVaults': 0,
 'numLiquidatingVaults': 0,
 'numLiquidationsCompleted': 0,
 'retainedCollateral': {'brand': {'boardId': 'board03446',
   'iface': 'Alleged: IbcATOM brand'},
  'value': 0},
 'totalCollateral': {'brand': {'boardId': 'board03446',
   'iface': 'Alleged: IbcATOM brand'},
  'value': 0},
 'totalCollateralSold': {'brand': {'boardId': 'board03446',
   'iface': 'Alleged: IbcATOM brand'},
  'value': 0},
 'totalDebt': {'brand': {'boardId': 'board0074',
   'iface': 'Alleged: IST brand'},
  'value': 0},
 'totalOverageReceived': {'brand': {'boardId': 'board0074',
   'iface': 'Alleged: IST brand'},
  'value': 0},
 'totalProceedsReceived': {'brand': {'boardId': 'board0074',
   'iface': 'Alleged: IST brand'},
  'value': 0},
 'totalShortfallReceived': {'brand': {'boardId': 'board0074',
   'iface': 'Alleged: IST brand'},
  'value': 0}}

In [34]:
IO_TESTING and _vaultFactory.governance(_fromBoard).current

{'Electorate': {'type': 'invitation',
  'value': {'brand': {'boardId': 'board03125',
    'iface': 'Alleged: Zoe Invitation brand'},
   'value': [{'description': 'questionPoser',
     'handle': {'boardId': None, 'iface': 'Alleged: InvitationHandle'},
     'installation': {'boardId': 'board05311',
      'iface': 'Alleged: BundleInstallation'},
     'instance': {'boardId': 'board00613',
      'iface': 'Alleged: InstanceHandle'}}]}},
 'LiquidationInstall': {'type': 'installation',
  'value': {'boardId': 'board01422', 'iface': 'Alleged: BundleInstallation'}},
 'LiquidationTerms': {'type': 'unknown',
  'value': {'AMMMaxSlippage': {'denominator': {'brand': {'boardId': 'board0074',
      'iface': 'Alleged: IST brand'},
     'value': 100},
    'numerator': {'brand': {'boardId': 'board0074',
      'iface': 'Alleged: IST brand'},
     'value': 30}},
   'MaxImpactBP': 50,
   'OracleTolerance': {'denominator': {'brand': {'boardId': 'board0074',
      'iface': 'Alleged: IST brand'},
     'value': 10

## Exploring `agoricNames`

In [28]:
IO_TESTING and _agoricNames.instance.keys()

dict_keys(['economicCommittee', 'amm', 'ammGovernor', 'VaultFactory', 'feeDistributor', 'Treasury', 'VaultFactoryGovernor', 'stakeFactory', 'Pegasus', 'reserve', 'reserveGovernor', 'psm', 'psmGovernor', 'interchainPool', 'ATOM-USD price feed'])

In [35]:
_agoricNames.installation.keys()

dict_keys(['centralSupply', 'mintHolder', 'walletFactory', 'contractGovernor', 'committee', 'binaryVoteCounter', 'amm', 'VaultFactory', 'feeDistributor', 'liquidate', 'stakeFactory', 'Pegasus', 'reserve', 'psm', 'interchainPool', 'scaledPriceAuthority', 'priceAggregator', 'econCommitteeCharter'])